In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from collections import Counter
import warnings
import gc
# from pandasgui import show
warnings.filterwarnings("ignore")

In [ ]:
pd.set_option('display.max_rows',5)   # 具体的行数或列数可自行设置
pd.set_option('display.max_columns',100)

In [ ]:
def get_base_info(x):
    return [i.split(':')[-1] for i in x.split(' ')]

def get_speed(x):
    return np.array([i.split(',')[0] for i in x], dtype='float16')

def get_eta(x):
    return np.array([i.split(',')[1] for i in x], dtype='float16')

def get_state(x):
    return np.array([(i.split(',')[2]) for i in x], dtype='int16')

def get_cnt(x):
    return np.array([i.split(',')[3] for i in x], dtype='int16')

In [ ]:
path = 'test.txt'

In [ ]:
df = pd.read_csv(path, sep=';', header=None)

In [ ]:
df['link'] = df[0].apply(lambda x: x.split(' ')[0])
df['label'] = -1
df['current_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[2]))
df['future_slice_id'] = df[0].apply(lambda x: int(x.split(' ')[3]))

In [ ]:
df['time_diff'] = df['future_slice_id'] - df['current_slice_id']

In [ ]:
df

In [ ]:
gc.collect()

In [ ]:
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 1 if x <= 200 else x)
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 2 if (200< x <= 230) else x)
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 3 if (230< x <= 340) else x)
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 4 if (340< x <= 470) else x)
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 5 if (470< x <= 520) else x)
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 6 if (520< x <= 588) else x)
df['current_slice_id'] = df['current_slice_id'].apply(lambda x: 7 if (588< x <= 720) else x)

In [ ]:
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 1 if x <= 200 else x)
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 2 if (200< x <= 230) else x)
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 3 if (230< x <= 340) else x)
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 4 if (340< x <= 470) else x)
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 5 if (470< x <= 520) else x)
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 6 if (520< x <= 588) else x)
df['future_slice_id'] = df['future_slice_id'].apply(lambda x: 7 if (588< x <= 720) else x)

In [ ]:
df

In [ ]:
for i in tqdm(range(1, 6)):
        df['his_info'] = df[i].apply(get_base_info)
        if i == 1:
            flg = 'current'
        else:
            flg = f'his_{(6 - i) * 7}'
        df['his_speed'] = df['his_info'].apply(get_speed)
        df[f'{flg}_speed_min'] = df['his_speed'].apply(lambda x: x.min())
        df[f'{flg}_speed_max'] = df['his_speed'].apply(lambda x: x.max())
        df[f'{flg}_speed_mean'] = df['his_speed'].apply(lambda x: x.mean())
        df[f'{flg}_speed_std'] = df['his_speed'].apply(lambda x: x.std())

        df['his_eta'] = df['his_info'].apply(get_eta)
        df[f'{flg}_eta_min'] = df['his_eta'].apply(lambda x: x.min())
        df[f'{flg}_eta_max'] = df['his_eta'].apply(lambda x: x.max())
        df[f'{flg}_eta_mean'] = df['his_eta'].apply(lambda x: x.mean())
        df[f'{flg}_eta_std'] = df['his_eta'].apply(lambda x: x.std())

        df['his_cnt'] = df['his_info'].apply(get_cnt)
        df[f'{flg}_cnt_min'] = df['his_cnt'].apply(lambda x: x.min())
        df[f'{flg}_cnt_max'] = df['his_cnt'].apply(lambda x: x.max())
        df[f'{flg}_cnt_mean'] = df['his_cnt'].apply(lambda x: x.mean())
        df[f'{flg}_cnt_std'] = df['his_cnt'].apply(lambda x: x.std())
        
        df['his_state'] = df['his_info'].apply(get_state)
        df[f'{flg}__state_min'] = df['his_state'].apply(lambda x: x.min())
        df[f'{flg}__state_max'] = df['his_state'].apply(lambda x: x.max())
        df[f'{flg}_state'] = df['his_state'].apply(lambda x: Counter(x).most_common()[0][0])
        df.drop([i, 'his_info', 'his_speed', 'his_eta', 'his_cnt', 'his_state'], axis=1, inplace=True)
        #df.drop([i, 'his_info', 'his_speed', 'his_eta', 'his_cnt'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
attr = pd.read_csv('attr.txt', sep='\t',
                       names=['link', 'length', 'direction', 'path_class', 'speed_class', 'LaneNum', 'speed_limit',
                              'level', 'width'], header=None)

In [ ]:
df['link'] = df['link'].apply(int)

In [ ]:
del df[0]

In [ ]:
test = df.merge(attr, on='link', how='left')

In [ ]:
test

In [ ]:
test

In [ ]:
import pickle
with open('knn.txt','rb')as f:
    clf = pickle.load(f)

In [ ]:
use_cols = [i for i in test.columns if i not in ['link', 'label', 'label_pred','level']]

In [ ]:
X_test = test[use_cols]

In [ ]:
X_test

In [ ]:
X_test = np.array(X_test)

In [ ]:
clf.predict(X_test[504879].reshape(1,-1))[0]

In [ ]:
ress = []
for i in tqdm(range(10)):
    res = clf.predict(X_test[100*i:100*(i+1)].reshape(100,-1))
    #print(res,' ',y_ver[i])
    #ress.append(res)
    print(res+1)

In [ ]:
ress = []
for i in tqdm(range(504891)):
    res = clf.predict(X_test[i].reshape(1,-1))[0]
    #print(res,' ',y_ver[i])
    ress.append(res)

In [ ]:
ans = np.array(ress)+1

In [ ]:
len(ress)

In [ ]:
ans_1 = test[['link','current_slice_id','future_slice_id','label']]

In [ ]:
ans_1

In [ ]:
ans_1['label'] = ans

In [ ]:
ans_1

In [ ]:
ans_1.to_csv('2020_10_31.csv', index=False, encoding='utf8')

In [ ]:
df_test = pd.read_csv(path, sep=';', header=None)

In [ ]:
ahh = df_test[['link','current_slice_id','future_slice_id','label']]

In [ ]:
ahh.to_csv('2020_10_31.csv', index=False, encoding='utf8')

In [ ]:
df_test['label'] = ans

In [ ]:
df_test

In [ ]:
df_test['link'] = df_test[0].apply(lambda x: x.split(' ')[0])
df_test['label'] = -1
df_test['current_slice_id'] = df_test[0].apply(lambda x: int(x.split(' ')[2]))
df_test['future_slice_id'] = df_test[0].apply(lambda x: int(x.split(' ')[3]))